In [10]:
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_openai import ChatOpenAI 
from typing import TypedDict, List

In [11]:
#Define the state schema
class ChatState(TypedDict):
    messages: List  # list of HumanMessage, SystemMessage, AIMessage

In [12]:
#Define user input node
def input_node(state):
    return state


In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama3-70b-8192",  # or 
    api_key=""
)


In [14]:
#Define chatbot node
def chatbot_node(state):

    # Add system prompt only once at beginning
    system_prompt = SystemMessage(content="You are a Math tutor chatbot.")
    messages = [system_prompt] + state["messages"]

    # LLM call
    response = llm.invoke(messages)

    # Add response to message history
    state["messages"].append(response)
    return state


In [15]:
#Define output node
def output_node(state):
    response = state["messages"][-1].content
    print("Bot:", response)
    return state

In [16]:
#Build LangGraph
graph = StateGraph(ChatState)
graph.add_node("input", input_node)
graph.add_node("chatbot", chatbot_node)
graph.add_node("output", output_node)

graph.set_entry_point("input")
graph.add_edge("input", "chatbot")
graph.add_edge("chatbot", "output")
graph.add_edge("output", END)


In [17]:
app = graph.compile()


In [ ]:
# Run chatbot
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        break

    state = {"messages": [HumanMessage(content=user_input)]}
    app.invoke(state)


Bot: Hey! I'm your math tutor chatbot! How can I help you with math today? Do you have a specific problem you're stuck on, or a concept you'd like to learn? Let me know and I'll do my best to assist you!
Bot: That's an easy one!

The answer is... 4!
Bot: I'm happy to help! However, I think there might be a slight misunderstanding. As a math tutor chatbot, I'm here to assist with math-related questions and topics. Ear infections are a medical topic and not directly related to math.

But, if you're curious, an ear infection, also known as otitis media, is a common condition that occurs when the middle ear becomes infected. It can cause symptoms such as ear pain, fever, and difficulty hearing. Ear infections are often caused by bacteria or viruses and can be treated with antibiotics or other medications.

If you have any math-related questions or topics you'd like to discuss, I'm here to help!
Bot: You're welcome! It seems like our conversation just started, though. Do you have any math q